In [1]:
from pynq import Overlay
#Overlay("base.bit").download()
# initialize camera from OpenCV
from pynq.drivers import Frame
import cv2
import math
# Output webcam image as JPEG
%pylab inline 
from IPython.display import clear_output
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
# monitor configuration: 640*480 @ 60Hz
from pynq.drivers import HDMI
from pynq.drivers.video import VMODE_640x480
hdmi_out = HDMI('out', video_mode=VMODE_640x480)
hdmi_out.start()


In [3]:
from pynq.iop import Robot_Controller
my_robot = Robot_Controller()

In [4]:
# monitor (output) frame buffer size
frame_out_w = 1920
frame_out_h = 1080
# camera (input) configuration
frame_in_w = 320 #640
frame_in_h = 240 #480

In [5]:
def update_shape(contours):
    approx = cv2.approxPolyDP(contours,0.01*cv2.arcLength(contours,True),True) 
    if len(approx) == 4:
        return "square"
            
    elif len(approx) > 8 and len(approx) < 12:
        return "cylinder"        
    else:
        return "invalid"

In [6]:
def update_box_points(contours):
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.array(box, dtype="int")
    return box

In [7]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
    
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect

In [8]:
def get_dimensions(pts,known_width):
    dimensions = {"height_px":None, "width_px":None, "w_In":None}
    
    # obtain a consistent order of the points and unpack them
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the bounding box, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    dimensions["width_px"] = maxWidth
    dimensions["w_In"] = maxWidth/known_width
              
    # compute the height of the bounding box, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dimensions["height_px"] = maxHeight
    return dimensions

In [9]:
def find_frame_pos(centerX):
    position = ''
    
    if (int(centerX) > 0) and (int(centerX) < 213):
        position = "left"  
    
    elif (int(centerX) > 213) and (int(centerX) < 426):
        position = "middle"
     
    else:    
        position ="right"   
    
    return position

In [10]:
def find_object_x_offset(centerX):
    # Find the x offset by taking the frame center (width/2) and subtract the current object's x-center.
    # This means that an object to the left of center will have a positive value and an object to the right
    # of center will have a negative value.  The offset should be at most abs(frame_in_w/2)
    return (frame_in_w/2) - centerX

In [11]:
def distance_to_camera(knownWidth, focalLength, percieved_width):
    # compute and return the distance from the maker to the camera
    return (knownWidth * focalLength) / percieved_width

In [12]:
def update_distance(known_width,focalLength, perWidth):
    inches = distance_to_camera(known_width, focalLength, perWidth)
    return inches

In [13]:
def center_to_object(xOffset):
    absX = abs(xOffset)
    if absX > 20:
        if xOffset < 0:
            my_robot.left()
            print(xOffset)
        else:
            my_robot.right()
            print(xOffset)
    elif absX <= 20:
        centered = True
   

In [14]:
def approach_object(distance):
    my_robot.release()
    for i in range(0,math.floor(distance)):
        my_robot.forward(0.05)
    
    my_robot.grip()
    

In [15]:
def get_can(position, distance):
    if not centered:
        center_to_object(position)
    else:
        approach_object(distance)
        
                
                
                
    

In [16]:
#Camera Calibration
#Known width/height of coke can
KNOWN_CAN_WIDTH          = 2.598  # inches
KNOWN_TARGET_WIDTH       = 13.937 # inches
KNOWN_CAN_PIXEL_WIDTH    = 133
KNOWN_TARGET_PIXEL_WIDTH = 344
KNOWN_CAN_DISTANCE       = 12 #empirically derived distance from lens to coke can
KNOWN_TARGET_DISTANCE    = 26 # inches
focalLength_can = (KNOWN_CAN_PIXEL_WIDTH * KNOWN_CAN_DISTANCE) / KNOWN_CAN_WIDTH
focalLength_target = (KNOWN_TARGET_PIXEL_WIDTH * KNOWN_TARGET_DISTANCE) / KNOWN_TARGET_WIDTH

In [17]:
#cap = cv2.VideoCapture(0)
shape =' '
results ={}
position=' '
distance = 0
cnt_length = 0
centered = False

# define range of red color in HSV
lower_red = np.array([17, 15, 100], dtype=np.uint8)
upper_red = np.array([50, 56, 200], dtype=np.uint8)
# define range of green color in HSV
lower_green = np.array([34, 50, 50], dtype=np.uint8)
upper_green = np.array([80, 220, 200], dtype=np.uint8)
# define range of blue color in HSV
lower_blue = np.array([92, 0, 0], dtype=np.uint8)
upper_blue = np.array([124, 255, 255], dtype=np.uint8)

In [18]:
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("Capture device is open: " + str(videoIn.isOpened()))

Capture device is open: True


In [ ]:
from pprint import pprint
try:
    while(True):
        # Capture frame-by-frame
        ret, frame = videoIn.read()
        if not ret:
            # Release the Video Device if ret is false
            videoIn.release()
            # Message to be displayed after releasing the device
            print ("Released Video Resource")
            break 
            
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hsv =  hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # Turn off the axis
        axis('off')
        
        # Threshold the HSV image to get only red colors
        mask_red = cv2.inRange(hsv, lower_red, upper_red)
        mask_green = cv2.inRange(hsv, lower_green, upper_green)
        mask_blue  = cv2.inRange(hsv, lower_blue, upper_blue)
        mask_zero = np.zeros(hsv.shape[:2],np.uint8)
        
        # RGB to Gray scale conversion
        img_gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
        # Noise removal with iterative bilateral filter(removes noise while preserving edges)
        noise_removal = cv2.bilateralFilter(img_gray,9,75,75)
        # Thresholding the image
        ret,thresh_image = cv2.threshold(noise_removal,0,255,cv2.THRESH_OTSU)
    
        # Applying Canny Edge detection
        canny_image = cv2.Canny(thresh_image,250,255)
        # Display Image
        canny_image = cv2.convertScaleAbs(canny_image)
    
        # dilation to strengthen the edges
        kernel = np.ones((3,3), np.uint8)
        # Creating the kernel for dilation
        dilated_image = cv2.dilate(canny_image,kernel,iterations=1)
    
        #find contour data
        __,contours,__ = cv2.findContours(dilated_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contours= sorted(contours, key = cv2.contourArea, reverse = True)[:1]
        for cnt in contours:
        # if the contour is not sufficiently large, ignore it
            if cv2.contourArea(cnt) < 100:
                continue
            shape = update_shape(cnt)
            if (shape == "square"):
                rect = cv2.minAreaRect(cnt)
                box = cv2.boxPoints(rect)
                box = np.array(box, dtype="int")
                cX = np.average(box[:, 0])
                # compute the center of the bounding box
                cX = np.average(box[:, 0])
                results = get_dimensions(box, KNOWN_TARGET_WIDTH)
                position = find_object_x_offset(cX)
                center_to_object(position)
                distance = update_distance(KNOWN_TARGET_WIDTH, focalLength_can, results["width_px"]) 
                approach_object(distance)
                print(results)
                print(position)
                print("dist from camera:", distance)
                if cv2.countNonZero(mask_blue):
                    print("Blue detected")
                    mask_green = mask_zero
                    mask_red = mask_zero
                    cv2.drawContours(frame,[box],0,(0,0,255),2)
                else:
                    continue
            elif (shape == "cylinder"):
                rect = cv2.minAreaRect(cnt)
                box = cv2.boxPoints(rect)
                box = np.array(box, dtype="int")
                cX = np.average(box[:, 0])
                # compute the center of the bounding box
                cX = np.average(box[:, 0])
                results = get_dimensions(box, KNOWN_CAN_WIDTH)
                position = find_frame_pos(cX)
                distance = update_distance(KNOWN_CAN_WIDTH, focalLength_can, results["width_px"]) 
                print(results)
                print(position)
                print("dist from camera:", distance)
                if cv2.countNonZero(mask_red):
                    print("Red detected")
                    mask_green = mask_zero
                    mask_blue = mask_zero
                    cv2.drawContours(frame,[box],0,(0,0,255),2)
                else:
                    continue
        imshow(frame)
        show()
        # Display the frame until new frame is available
        clear_output(wait=True)
#         break
except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    print("Released Video Resource")

In [ ]:
while(True):
    # Capture frame-by-frame
    ret, frame = videoIn.read()
    if not ret:
        # Release the Video Device if ret is false
        videoIn.release()
        # Message to be displayed after releasing the device
        print ("Released Video Resource")
        break   
    # Our operations on the frame come here
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Turn off the axis
    axis('off')
    # Threshold the HSV image to get only red colors
    mask_red = cv2.inRange(hsv, lower_red, upper_red)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_blue  = cv2.inRange(hsv, lower_blue, upper_blue)
    mask_zero = np.zeros(hsv.shape[:2],np.uint8)
    
    # RGB to Gray scale conversion
    img_gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    # Noise removal with iterative bilateral filter(removes noise while preserving edges)
    noise_removal = cv2.bilateralFilter(img_gray,9,75,75)
    # Thresholding the image
    ret,thresh_image = cv2.threshold(noise_removal,0,255,cv2.THRESH_OTSU)

    # Applying Canny Edge detection
    canny_image = cv2.Canny(thresh_image,250,255)
    # Display Image
    canny_image = cv2.convertScaleAbs(canny_image)

    # dilation to strengthen the edges
    kernel = np.ones((3,3), np.uint8)
    # Creating the kernel for dilation
    dilated_image = cv2.dilate(canny_image,kernel,iterations=1)
   
    
    #find contour data
    __,contours,__ = cv2.findContours(dilated_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours= sorted(contours, key = cv2.contourArea, reverse = True)[:1]
    pt = (180, 3 * frame.shape[0] // 4)
    for cnt in contours:
        # if the contour is not sufficiently large, ignore it
        if cv2.contourArea(cnt) < 100:
            continue
            
        shape = update_shape(cnt)
        if (shape == "square"):
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.array(box, dtype="int")
            cX = np.average(box[:, 0])
            # compute the center of the bounding box
            cX = np.average(box[:, 0])
            results = get_dimensions(box, KNOWN_TARGET_WIDTH)
            position = find_object_x_offset(cX)
            center_to_object(position)
            distance = update_distance(KNOWN_TARGET_WIDTH, focalLength_can, results["width_px"]) 
            approach_object(distance)
            print(results)
            print(position)
            print("dist from camera:", distance)
            if cv2.countNonZero(mask_blue):
                print("Blue detected")
                mask_green = mask_zero
                mask_red = mask_zero
                cv2.drawContours(frame,[box],0,(0,0,255),2)
            else:
                continue

        elif (shape == "cylinder"):
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.array(box, dtype="int")
            cX = np.average(box[:, 0])
            # compute the center of the bounding box
            cX = np.average(box[:, 0])
            results = get_dimensions(box, KNOWN_CAN_WIDTH)
            position = find_object_x_offset(cX)
            distance = update_distance(KNOWN_CAN_WIDTH, focalLength_can, results["width_px"])
            break
            
            '''
            if not centered:
                center_to_object(position)
            else:
                approach_object(distance)
                break
            '''
        # get_can(position,distance)  
        print(results)
        print(position)
        print("dist from camera:", distance)
        if cv2.countNonZero(mask_red):
            print("Red detected")
            mask_green = mask_zero
            mask_blue = mask_zero
            cv2.drawContours(frame,[box],0,(0,0,255),2)
        else:
            continue
            
 
  

In [ ]:
# Capture webcam image
import numpy as np

ret, frame_vga = videoIn.read()

# Display webcam image via HDMI Out
if (ret):
    frame_1080p = np.zeros((1080,1920,3)).astype(np.uint8)       
    frame_1080p[0:480,0:640,:] = frame_vga[0:480,0:640,:]
    hdmi_out.frame_raw(bytearray(frame_1080p.astype(np.int8).tobytes()))
else:
    raise RuntimeError("Failed to read from camera.")

In [ ]:
# Output webcam image as JPEG
%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np
plt.imshow(frame_vga[:,:,[2,1,0]])
plt.show()

In [ ]:
import cv2

np_frame = frame_vga

face_cascade = cv2.CascadeClassifier(
                        './data/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(
                        './data/haarcascade_eye.xml')

gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

for (x,y,w,h) in faces:
    cv2.rectangle(np_frame,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = np_frame[y:y+h, x:x+w]

    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

In [ ]:
# Output OpenCV results via HDMI
frame_1080p[0:480,0:640,:] = frame_vga[0:480,0:640,:]
hdmi_out.frame_raw(bytearray(frame_1080p.astype(np.int8).tobytes()))

In [ ]:
# Output OpenCV results via matplotlib
%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np
plt.imshow(np_frame[:,:,[2,1,0]])
plt.show()

In [ ]:
videoIn.release()
hdmi_out.stop()
del hdmi_out